# S02 — Map-Matching (Safety-Oriented)

Привязываем точки к дорожному графу OSM с **проверкой направления**.  
Если граф **есть** локально (`artifacts/osm_graph.graphml`) — используем его.  
Если нет — скачиваем bbox по данным (нужен интернет), сохраняем для репро.

In [ ]:
%run ./S00_setup.ipynb

In [ ]:
# 1) Загрузка исходных данных
read_kwargs = dict(sep=",", engine="c",
                   dtype={"randomized_id":"int64","lat":"float64","lng":"float64","alt":"float64","spd":"float64","azm":"float64"},
                   header=0)
try:
    sample = pd.read_csv(DATA_PATH, nrows=5, **read_kwargs)
    exp = ["randomized_id","lat","lng","alt","spd","azm"]
    if list(sample.columns[:6]) != exp:
        read_kwargs.update({"header":None,"names":exp})
except Exception:
    read_kwargs.update({"header":None,"names":["randomized_id","lat","lng","alt","spd","azm"]})
df = pd.read_csv(DATA_PATH, **read_kwargs)
print("Loaded:", df.shape)

In [ ]:
# 2) Определение bbox по данным
lat_min, lat_max = df["lat"].min(), df["lat"].max()
lng_min, lng_max = df["lng"].min(), df["lng"].max()
pad = 0.003  # небольшой запас
bbox = (lat_max+pad, lat_min-pad, lng_max+pad, lng_min-pad)  # (north, south, east, west)
print("BBox:", bbox)

In [ ]:
# 3) Загрузка/скачивание OSM графа
if ox is None:
    print("osmnx не установлен. Для полного map-matching установите osmnx и перезапустите.")
else:
    if GRAPH_PATH.exists():
        G = ox.load_graphml(GRAPH_PATH)
        print("Граф загружен из", GRAPH_PATH)
    else:
        print("Граф не найден локально. Пытаемся скачать с OSM (требуется интернет)...")
        G = ox.graph_from_bbox(bbox=bbox, simplify=True, network_type="drive")
        ox.save_graphml(G, GRAPH_PATH)
        print("Сохранён:", GRAPH_PATH)

    # Подготовка удобных структур
    G_und = ox.convert.to_undirected(G)
    # Создадим GeoDataFrames (если доступен geopandas)
    edges_gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
    nodes_gdf = ox.graph_to_gdfs(G, nodes=True, edges=False)
    print("Edges:", edges_gdf.shape, "Nodes:", nodes_gdf.shape)

In [ ]:
# 4) Map-matching: для каждой точки — ближайшее ребро + проверка направления
if ox is not None:
    from shapely.geometry import Point
    import numpy as np

    # Преобразуем точки в список
    pts = list(zip(df["lng"].values, df["lat"].values))
    # Находим ближайшие ребра (u,v,key) по координатам
    # osmnx 1.x: nearest_edges(G, X, Y, return_dist=True)
    ne = ox.distance.nearest_edges(G, X=[p[0] for p in pts], Y=[p[1] for p in pts], return_dist=True)
    uvk_list, dist_list = ne[0], ne[1]

    # Функция: азимут ребра в ближайшей к точке позиции (приближение: берем общий азимут linestring)
    def edge_bearing(row):
        geom = row.geometry
        try:
            x0,y0 = geom.coords[0]
            x1,y1 = geom.coords[-1]
            return bearing_deg(y0,x0,y1,x1)
        except Exception:
            return np.nan

    edges_gdf["edge_bearing"] = edges_gdf.apply(edge_bearing, axis=1)
    edges_gdf["highway"] = edges_gdf["highway"].apply(lambda x: x[0] if isinstance(x, (list, tuple)) and x else x)
    edges_gdf["oneway"] = edges_gdf["oneway"].fillna(False).astype(bool)

    # Собираем таблицу соответствий
    mm = pd.DataFrame({
        "randomized_id": df["randomized_id"].values,
        "lat": df["lat"].values,
        "lng": df["lng"].values,
        "alt": df["alt"].values,
        "spd": df["spd"].values,
        "azm": df["azm"].values,
        "u": [uvk[0] for uvk in uvk_list],
        "v": [uvk[1] for uvk in uvk_list],
        "key": [uvk[2] for uvk in uvk_list],
        "dist2edge": dist_list
    })

    # Мержим с атрибутами ребра
    edges_key = edges_gdf.reset_index()[["u","v","key","highway","oneway","edge_bearing","length"]]
    mm = mm.merge(edges_key, on=["u","v","key"], how="left")

    # Угловое несоответствие
    mm["bearing_dev"] = np.abs(((mm["azm"] - mm["edge_bearing"] + 180) % 360) - 180)
    mm["match_ok"] = (mm["bearing_dev"] <= CONFIG["ANGLE_TOL_DEG"])

    # Примерные расстояния до узлов: до ближайшего узла по геодезии
    # (более точно можно проецировать на линию и мерить до узлов; оставим приближение)
    # Выберем ближайший узел к точке:
    nn = ox.distance.nearest_nodes(G, X=mm["lng"].values, Y=mm["lat"].values, return_dist=True)
    mm["dist2node"] = nn[1]

    if "geometry" in mm.columns:
        mm = mm.drop(columns=["geometry"])

    # Приведем типы и сохраним
    mm.to_parquet(MATCHED_PARQUET, index=False, engine="pyarrow")
    print("Saved matched points to:", MATCHED_PARQUET)
else:
    print("Пропускаем map-matching: osmnx не установлен. Дальше будут доступны H3-агрегации.")